# helpers

> This module contains all the various helper functions used in the other modules.

In [ ]:
#| default_exp helpers

In [ ]:
#| export
import re

In [ ]:
#| hide
from nbdev.showdoc import *

:::{.callout-important}
Note: This notebook contains a large collection of profane and offensive language to use as a word filter. It is not recommended for children or the highly sensitive.
:::

In [ ]:
#| hide
# def quick_size_estimation(
#     ds: Dataset,
#     num_proc: int,
#     batch_size: int,
#     content_key:str ="text"
# ) -> int:
#     if len(ds) == 0:
#         return 0
#     rng = default_rng(1991)
#     subset_size = min(10000, len(ds))
#     indices = rng.choice(len(ds), size=subset_size, replace=False, shuffle=False)
#     partial_ds = ds.select(indices)
#     ratio = float(len(ds)) / float(subset_size)

#     partial_ds = partial_ds.map(
#         get_size_per_example,
#         batched=True, 
#         num_proc=num_proc,
#         batch_size=batch_size,
#         input_columns=[content_key],
#         remove_columns=partial_ds.column_names,
#     )
#     len_bytes = sum(partial_ds["bytes_len"])
#     return len_bytes * ratio

In [ ]:
#| export
def get_words(
    text: str, # the text to extract words from
) -> list:
    """custom regex to extract all the words in a string"""
    return re.findall(r'\w+', text.lower())

In [ ]:
#| export
# Built from native speakers, with inspiration from
# https://github.com/zacanger/profane-words
# and
# https://github.com/thisandagain/washyourmouthoutwithsoap/blob/develop/data/build.json
# and
# https://github.com/LDNOOBW/List-of-Dirty-Naughty-Obscene-and-Otherwise-Bad-Words


english_flagged_words = [
    "anal",
    # "bareback", # not sure about this one
    "bbw",
    "bdsm",
    "blowjob",
    "blowjobs",
    "brazzers",
    "bukkake",
    "camgirl",
    "camwhore",
    "cocksucking",
    # "cougar", # not sure about this one
    "creampie",
    "cuckold",
    "cum",
    "cumming",
    "cums",
    "cumshot",
    "cumshots",
    "cumslut",
    "cunnilingus",
    "deepthroat",
    "deepthroating",
    "dildo",
    "dildos",
    "dogging",
    "doggystyle",
    # "dominatrix", # not sure about this one
    "erotic",
    "fellatio",
    "femdom",
    "fingering",
    "fisting",
    "footjob",
    "gangbang",
    "handjob",
    "hentai",
    "horney",
    "horniest",
    "horny",
    "jism",
    "jizz",
    "lolli",
    "lolling",
    "masterbating",
    "masturbate",
    "masturbating",
    "masturbation",
    "milf",
    "orgies",
    "orgy",
    "pegging",
    "porn",
    "pornhub",
    "porno",
    "pornos",
    "pornstar",
    "pornstars",
    "redtube",
    "rimming",
    "slutty",
    # "squirting", # not sure about this one
    "strapon",
    "threesome",
    "vibrator",
    "xhamster",
    "xnxx",
    "xvideos",
    "xxx",
    "youporn",
]


flagged_words = {
    "ar": english_flagged_words
    + [
        "إباحي",
        "احتلام",
        "است",
        "استمناء",
        "اغتصاب",
        "أورغازم",
        "إيروتيك",
        "أير",
        "بز",
        "بزاز",
        "بظر",
        "بورن",
        "بيضان",
        "مص",
        "ثدي",
        "جماع",
        "جنس",
        "حلمة",
        "خلاعة",
        "خنثي",
        "خول",
        "دعارة",
        "زب",
        "سحاق",
        "سحاقية",
        "سكس",
        "سيكس",
        "شاذ",
        "شبق",
        "شرج",
        "شرموطة",
        "شهواني",
        "شهوة",
        "طيز",
        "عادة السرية",
        "عاهرة",
        "عرص",
        "فاسقة",
        "فرج",
        "قحبة",
        "قضيب",
        "كس",
        "لحس",
        "لعق",
        "لواط",
        "لوطي",
        "مؤخرة",
        "متناك",
        "متناكة",
        "مومس",
        "مثير",
        "مص",
        "مضاجعة",
        "مفلقسة",
        "مني",
        "مهتاج",
        "نشوة",
        "نكاح",
        "نيك",
    ],
    "bn": english_flagged_words
    + [
        "আঙ্গুলি করা",
        "আচোদা",
        "খানকি",
        "খানকি মাগি",
        "গান্ড মারানো",
        "গুদ মারানি",
        "চুচুক",
        "চোদ",
        "চোদনা",
        "চোদা",
        "চোদা বোন",
        "চোদাচুদি",
        "জারজ",
        "নাঙ্গ",
        "নেংটা",
        "পর্ণহাব",
        "পর্ন",
        "পর্নস্টার",
        "পর্নোগ্রাফি",
        "পোঁদ",
        "পোঁদ মারানি",
        "পোদ মারানি",
        "বাঁড়া",
        "বানচোদ",
        "বেশ্যা",
        "বেশ্যার ছেলে",
        "বোকাচোদা",
        "ভগ",
        "মা চোদা",
        "মাগী",
        "মাদারচোদ",
        "মুখে নিবি",
        "মোরগ",
        "রেন্ডি",
        "শিশ্ন",
        "স্তন",
        "স্তনবৃন্ত",
        "হস্তমৈথুন",
    ],
    "ca": english_flagged_words
    + [
        "cagarro",
        "cagarros",
        "cipote",
        "cipotes",
        "collons",
        "colló",
        "consolador",
        "consoladors",
        "cony",
        "conys",
        "corre's",
        "corre't",
        "corregut",
        "cunillingus",
        "córrer-se",
        "escorreguda",
        "escorregudes",
        "escorregut",
        "escrot",
        "escrots",
        "escórre's",
        "escórre't",
        "escórrer-se",
        "mamada",
        "mamadera",
        "mamaderes",
        "mamades",
        "masturba",
        "masturbacions",
        "masturbació",
        "masturbant",
        "masturbar",
        "masturbar-se",
        "masturbat",
        "masturbats",
        "masturbes",
        "orgasme",
        "orgasmes",
        "ou",
        "ous",
        "palla",
        "palles",
        "pornografia",
        "semen",
        "semens",
        "verga",
        "vergues",
        "xxx",
    ],
    "en": english_flagged_words,
    "es": english_flagged_words
    + [
        "chupar el coño",
        "chupar la concha",
        "chupar la polla",
        "chupar la verga",
        "comer el coño",
        "comer la concha",
        "comer la polla",
        "comer la verga",
        "coprofagía",
        "correrse",
        "cunillingus",
        "fagging",
        "felación",
        "felching",
        "follada",
        "follador de culo",
        "folladores",
        "fudge packer",
        "hacer una paja",
        "hacerse una paja",
        "hore",
        "kock",
        "macizorra",
        "madre folladora",
        "mamada",
        "perro follador",
        "pisser",
        "pornografía",
        "sado",
        "sadomasoquismo",
        "sadomasoquista",
        "sexo anal",
        "skank",
        "smegma",
        "x clasificado",
    ],
    "eu": english_flagged_words + [],
    "fr": english_flagged_words
    + [
        "baiseurs",
        "baiseur",
        "baiseuse",
        "baiseuses",
        "branlette",
        "branlettes",
        "branleuse",
        "branleuses",
        "cunillingus",
        "cunilingus",
        "enculée",
        "enculées",
        "enculation",
        "enculations",
        "enculement",
        "enculements",
        "fellation",
        "fellations",
        "porno",
        "pornos",
        "pornographie",
        "pornographique",
        "pornographiques",
        "salope",
        "salopes",
        "suceuse",
        "suceuses",
        "xxx",
    ],
    "hi": english_flagged_words
    + [
        "अंडकोश की थैली",
        "एक्स रेटेड",
        "ओगाज़्म",
        "कामोद्दीपक चित्र",
        "कालीन का चूरा",
        "कून",
        "कॉक",
        "गेंद का थैला",
        "चाकलेट का रंग",
        "चूची",
        "चूतड़",
        "झटका बंद",
        "ठगना पैकर",
        "डिल्डो",
        "नितंब",
        "पिछाड़ी",
        "पीड़न कामुक",
        "पॉर्न",
        "फटना",
        "फूहड़",
        "बट",
        "बहुत मदहोश",
        "बेल अंत",
        "भगवान-शापित",
        "भगशेफ",
        "माँ कमीने",
        "मुखमैथुन",
        "मुर्गा चूसने वाला",
        "रक्तरंजित",
        "लेबिया",
        "वहशी",
        "वहशीता",
        "वैंग",
        "शिश्नमल",
        "संभोग सुख",
        "सह शॉट",
        "सींग का बना हुआ",
        "होर",
        "घपा घप",
        "चुदाई",
        "चुदक्कड़",
    ],
    "id": english_flagged_words
    + [
        "bokep",
        "coli",
        "colmek",
        "grepe",
        "horni",
        "janda",
        "jembut",
        "jilat memek",
        "jilmek",
        "kontol",
        "masturbasi",
        "memek",
        "ngentot",
        "ngewe",
        "peju",
        "pepek",
        "pornografi",
        "sange",
        "sepong",
        "tusbol",
    ],
    "pt": english_flagged_words
    + [
        "balalao",
        "bate uma",
        "beijo grego",
        "boceta",
        "boquete",
        "buceta",
        "caralho",
        "chochota",
        "coito",
        "cona",
        "consolo",
        "corno",
        "cu",
        "dar a bunda",
        "dar o rabo",
        "dildo",
        "dildos",
        "esporrar",
        "estrovenga",
        "felação",
        "filho da puta",
        "filhos da puta",
        "gozada",
        "jeba",
        "perereca",
        "pica",
        "piru",
        "porno",
        "pornografia",
        "pornô",
        "porra",
        "prostituta",
        "pube",
        "punheta",
        "punheteiro",
        "putaria",
        "queca",
        "sexo",
        "siririca",
        "tesão",
        "trepada",
        "verga",
        "vibrador",
        "xana",
        "xochota",
        "xoxota",
    ],
    "ur": english_flagged_words
    + [
        "انگلی کرنا",
        "ایکس ریٹیڈ",
        "بلو جاب",
        "بٹ",
        "جھٹکا بند",
        "دلڈو",
        "رنڈی",
        "سلٹ",
        "سکلیرا",
        "سیڈسٹ",
        "سیکس بم",
        "شہوانی",
        "شہوت انگیز",
        "فحش نگاری",
        "لن",
        "لنڈ",
        "لنڈ چوسنے والا",
        "لوڑہ",
        "ماں کمینے",
        "مشت زنی",
        "ممے",
        "مٹھ",
        "مٹھی",
        "ویشیا",
        "پورن",
        "پھدی",
        "پیگنگ",
        "چدائ",
        "چدک",
        "چوت",
        "چودنا",
        "چوچی",
        "کسبی",
        "کسنگ",
        "گانڈ",
        "گدا",
        "ہینڈ جاب",
    ],
    "vi": english_flagged_words
    + [
        "cặc",
        "lồn",
        "địt",
        "buồi",
        "đụ",
        "nứng",
        "phò",
        "bú lol",
        "bú lon",
        "bú cac",
        "bú cak",
        "bú cax",
        "bú buoi",
        "bú mút",
        "húp lol",
        "húp lon",
        "húp sò",
        "nung lol",
        "nung lon",
        "nug lon",
        "nung cac",
        "nung buoi",
        "thèm dit",
        "thèm lon",
        "thèm lol",
        "thèm buoi",
        "thèm dit",
        "thèm chich",
        "thèm chịch",
        "thèm cax",
        "thèm cak",
        "móc lol",
        "móc lon",
        "buscu",
        "lol non",
        "bướm non",
        "vú to",
        "vú bự",
        "lon to",
        "phim sex",
        "phim xex",
        "phim xxx",
    ],
    "zh": english_flagged_words
    + [
        "三级片",
        "乳交",
        "卖B",
        "卖比",
        "卖淫",
        "口交",
        "姣西",
        "打炮",
        "打飞机",
        "抽插",
        "招妓",
        "插你",
        "插死你",
        "淫水",
        "肉壁",
        "肉棍子",
        "肉缝",
        "肏",
        "肛交",
    ],
}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()